In [2]:
from dotenv import load_dotenv
load_dotenv()

True

Gemini 
- 2.0 flash gratuita pero limitada
anthropic(Claude) - gratuita pero limitada
- 
OpenAI api paga

# Componentes

## Prompts

In [3]:
from langchain.prompts import PromptTemplate

promt_template = PromptTemplate.from_template(
    "Tell me a {adejective} joke about {topic}."
)

promt_template.format(
    adejective="funny",
    topic="Python"
)

'Tell me a funny joke about Python.'

## Chain

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
promt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("user", "{input}"),
    ]
)
chain = promt | llm
chain.invoke({"input": "Tell me a funny joke about Python."}).content

"Why did the Python say goodbye to the C++ programmer?\n\nBecause he didn't want to get bitten by a class!"

Branching

In [5]:

from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

# Define prompt templates for different feedback types
positive_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Generate a thank you note for this positive feedback: {feedback}."),
    ]
)

negative_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Generate a response addressing this negative feedback: {feedback}."),
    ]
)

neutral_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a request for more details for this neutral feedback: {feedback}.",
        ),
    ]
)

escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a message to escalate this feedback to a human agent: {feedback}.",
        ),
    ]
)

# Define the feedback classification template
classification_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Classify the sentiment of this feedback as positive, negative, neutral, or escalate: {feedback}."),
    ]
)

# Define the runnable branches for handling feedback
branches = RunnableBranch(
    (
        lambda x: "positive" in x,
        positive_feedback_template | model | StrOutputParser()  # Positive feedback chain
    ),
    (
        lambda x: "negative" in x,
        negative_feedback_template | model | StrOutputParser()  # Negative feedback chain
    ),
    (
        lambda x: "neutral" in x,
        neutral_feedback_template | model | StrOutputParser()  # Neutral feedback chain
    ),
    escalate_feedback_template | model | StrOutputParser()
)

# Create the classification chain
classification_chain = classification_template | model | StrOutputParser()

# Combine classification and response generation into one chain
chain = classification_chain | branches

# Run the chain with an example review
# Good review - "The product is excellent. I really enjoyed using it and found it very helpful."
# Bad review - "The product is terrible. It broke after just one use and the quality is very poor."
# Neutral review - "The product is okay. It works as expected but nothing exceptional."
# Default - "I'm not sure about the product yet. Can you tell me more about its features and benefits?"

review = "he product is excellent. I really enjoyed using it and found it very helpful."
result = chain.invoke({"feedback": review})

# Output the result
print(result)


Okay, here's a message to escalate positive feedback to a human agent, keeping in mind the "positive" aspect:

**Option 1 (Short & Sweet):**

> "Escalating to a human agent to share positive feedback received. Please review and consider for recognition/further action."

**Option 2 (Slightly More Detailed):**

> "Routing this positive feedback to a human agent for review.  This feedback highlights [mention a specific area if possible, e.g., excellent customer service, product innovation, etc.].  Please assess for potential rewards, recognition, or further action."

**Option 3 (Focus on Impact):**

> "Escalating this exceptionally positive feedback to a human agent. This feedback could be valuable for [mention potential use, e.g., employee performance reviews, marketing materials, product development]. Please review and determine appropriate next steps."

**Key Considerations When Choosing:**

*   **Your Internal Process:** Does your company have a specific process for handling positive 

## Memoria 

In [6]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Eres un asistente especializado en {abillity}. Puedes responder preguntas sobre {abillity}"),  # Changed from set to tuple
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),  # Changed from set to tuple
    ]
)
runnable = prompt | model

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    runnable=runnable,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [8]:
with_message_history.invoke(
        {"abillity":"ciencia de datos", "input":"Quero aprender ciencia de dados."},
        config={"configurable": {"session_id": "12345"}},
).content

'¡Excelente elección! La ciencia de datos es un campo fascinante y en constante crecimiento. Para ayudarte a empezar, te propongo una hoja de ruta estructurada y algunos recursos clave.\n\n**1. Fundamentos:**\n\n*   **Matemáticas:**\n    *   **Álgebra Lineal:** Vectores, matrices, operaciones matriciales, autovalores y autovectores. Esencial para entender algoritmos de machine learning.\n    *   **Cálculo:** Derivadas, integrales, optimización. Importante para entender cómo funcionan los algoritmos de optimización en machine learning.\n    *   **Estadística y Probabilidad:** Estadística descriptiva, distribuciones de probabilidad, pruebas de hipótesis, intervalos de confianza. Fundamental para el análisis de datos y la inferencia estadística.\n*   **Programación:**\n    *   **Python:** El lenguaje más popular en ciencia de datos. Aprende la sintaxis básica, estructuras de datos, funciones, clases y módulos.\n    *   **R:** Otro lenguaje popular, especialmente en estadística.\n*   **Bas

In [9]:
with_message_history.invoke(
        {"abillity":"ciencia de datos", "input":"Como?"},
        config={"configurable": {"session_id": "12345"}},
).content

'Para que pueda ayudarte de la mejor manera, necesito que me especifiques un poco más tu pregunta.  ¿A qué te refieres con "¿Cómo?"?  Podría ser:\n\n*   **¿Cómo empiezo a aprender ciencia de datos?** (Si necesitas una guía más detallada sobre los primeros pasos).\n*   **¿Cómo instalo Python y las bibliotecas necesarias?** (Si necesitas ayuda con la configuración del entorno).\n*   **¿Cómo encuentro conjuntos de datos para practicar?** (Si necesitas recursos para encontrar datos).\n*   **¿Cómo abordo un proyecto de ciencia de datos?** (Si necesitas una guía sobre el flujo de trabajo).\n*   **¿Cómo aprendo un tema específico (por ejemplo, álgebra lineal, estadística, machine learning)?** (Si necesitas recursos para un tema en particular).\n\nCuanto más específica sea tu pregunta, mejor podré ayudarte.  ¡No dudes en darme más detalles!'

## Tools

Solo tiene procesado hasta lo que se entreno, por lo que se pueden usar tools para buscar informacion en otras fuentes 

### Wikipedia

In [10]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper()
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

print("Nombre de herramienta:", wiki_tool.name) 
print("Descripción:", wiki_tool.description) 
print("Argumentos:", wiki_tool.args) 

Nombre de herramienta: wikipedia
Descripción: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
Argumentos: {'query': {'description': 'query to look up on wikipedia', 'title': 'Query', 'type': 'string'}}


In [11]:
wiki_tool.run("LangChain")  

'Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n\n\nPage: Milvus (vector database)\nSummary: Milvus is a distributed vector database developed by Zilliz. It is available as both open-source software and a cloud service.\nMilvus is an open-source project under LF AI & Data Foundation distributed under the Apache License 2.0.\n\nPage: Intelligent agent\nSummary: In artificial intelligence, an intelligent agent is an entity that perceives its environment, takes actions autonomously to achieve goals, and may improve its performance through machine learning or by acquiring knowledge. Leading AI textbooks define artificial intelligence as the "study and design of intelligent agents," emphasiz

### Servicio Tavily

https://app.tavily.com/home

In [12]:
from langchain_community.tools.tavily_search import TavilySearchResults

searchTavily = TavilySearchResults()
searchTavily.invoke("Que es LangChain?")

C:\Users\Socius\AppData\Local\Temp\ipykernel_5928\3392196689.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  searchTavily = TavilySearchResults()


[{'title': '¿Qué es LangChain? - Google Cloud',
  'url': 'https://cloud.google.com/use-cases/langchain?hl=es-419',
  'content': '# ¿Qué es LangChain?\n\nLangChain es una plataforma de lenguaje de programación que permite a los desarrolladores construir y conectar modelos para acceder, transformar y compartir datos sin problemas. Ofrece un entorno potente y versátil para el desarrollo de modelos, lo que permite la manipulación de estructuras de datos, la encadenación de modelos y la incorporación de agentes externos como LaMDA. [...] En su núcleo, LangChain aprovecha una arquitectura distribuida que permite un procesamiento eficiente y escalable de datos de lenguaje. Emplea un diseño basado en microservicios, en el que cada cadena se ejecuta como un servicio independiente, lo que facilita la implementación y administración flexibles. Esta arquitectura permite una integración continua con servicios externos, incluidos los LLM y las fuentes de datos basadas en la nube. [...] LangChain fun

In [13]:
#Lista de herramientas
tools = [wiki_tool, searchTavily]

## Agents

Son agentes de software autnomas que interactuan y responder dinamicamente con los usuarios y el entorno
- Capaces dde realizar acciones
- Utilizan y orquestan chains
- Mantiene contexto y memoria

HUB: Almacen de promts de la comunidad

In [27]:
from langchain import hub

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

In [28]:
system_prompt = "Eres un asistente de chat amigable y servicial, especializado en ayudar con información sobre Azure DevOps. Tu principal tarea es responder preguntas sobre el contenido de la **Wiki del proyecto Prueba-MCP** de Azure DevOps. Por favor, responde siempre en **español** y mantén un tono **simpático y conversacional**. Para responder a las preguntas, utilizarás tus herramientas para **buscar en la Wiki de Azure DevOps**. Es fundamental que asumas que toda la información relevante para las preguntas de los usuarios se encuentra dentro del **proyecto Prueba-MCP** y puede estar en **cualquier página de su Wiki**. Cuando un usuario te pregunte algo, tu primer paso debe ser buscar la respuesta en la Wiki utilizando alguna de las herramientas de búsqueda de información que tengas disponible para la Wiki de Azure DevOps. Si la información no está directamente disponible en la Wiki o si necesitas más detalles, puedes solicitar aclaraciones al usuario o sugerirle que la información podría no estar documentada. Si el usuario te pide algo que no está relacionado con la Wiki de Azure DevOps o las herramientas que tienes para interactuar con Azure DevOps, hazle saber amablemente que tu función principal es ayudarte con la Wiki y Azure DevOps."

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{input}"),
])
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Eres un asistente de chat amigable y servicial, especializado en ayudar con información sobre Azure DevOps. Tu principal tarea es responder preguntas sobre el contenido de la **Wiki del proyecto Prueba-MCP** de Azure DevOps. Por favor, responde siempre en **español** y mantén un tono **simpático y conversacional**. Para responder a las preguntas, utilizarás tus herramientas para **buscar en la Wiki de Azure DevOps**. Es fundamental que asumas que toda la información relevante para las preguntas de los usuarios se encuentra dentro del **proyecto Prueba-MCP** y puede estar en **cualquier página de su Wiki**. Cuando un usuario te pregunte algo, tu primer paso debe ser buscar la respuesta en la Wiki utilizando alguna de las herramientas de búsqueda de información que tengas disponi

In [31]:
prompt = hub.pull("hwchase17/openai-functions-agent")
promt


c:\Users\Socius\Desktop\Langchain\langchain practica\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [36]:
system_prompt = "Eres un asistente de chat amigable y servicial, que responde a la pregunta del usuario: {input}"
prompt_template = ChatPromptTemplate.from_template(system_prompt)

prompt = prompt_template.invoke({"input": "{input}"})

In [47]:
prompt = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("placeholder", "{msgs}") # <-- This is the changed part
],

    ("tools", tools))

TypeError: ChatPromptTemplate.__init__() takes 2 positional arguments but 3 were given

In [44]:
from langchain.agents import create_tool_calling_agent, create_react_agent, create_structured_chat_agent
from langchain.agents import AgentExecutor

agent = create_structured_chat_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

agent_executor.invoke(
    {
        "input": "Hola!",
    })

ValueError: Prompt missing required variables: {'tools', 'agent_scratchpad', 'tool_names'}

In [16]:
# Usando els ervicio de wikipedia
agent_executor.invoke({"input": "Quiero aprender sobre LangChain.",})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'LangChain'}`


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Milvus (vector database)
Summary: Milvus is a distributed vector database developed by Zilliz. It is available as both open-source software and a cloud service.
Milvus is an open-source project under LF AI & Data Foundation distributed under the Apache License 2.0.

Page: Intelligent agent
Summary: In artificial intelligence, an intelligent agent is an entity that perceives its environment, takes actions autonomously to achieve goals, and may improve its performance through machine learning or by acquiring knowledge. Leading AI textbooks defin

{'input': 'Quiero aprender sobre LangChain.',
 'output': 'LangChain es un marco de software que ayuda a facilitar la integración de grandes modelos de lenguaje (LLM) en aplicaciones. Sus casos de uso incluyen el análisis y resumen de documentos, chatbots y análisis de código.'}

In [17]:
# Usando el servicio de search
agent_executor.invoke({"input": "Puedes usar la herramienta tavily para buscar cual es la temperatura en buenos aires argentina?",})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'temperatura en buenos aires argentina'}`


[{'title': 'Previsión meteorológica de tres días para Buenos Aires ...', 'url': 'https://www.accuweather.com/es/ar/buenos-aires/7894/weather-forecast/7894', 'content': 'Temperatura. Previsión por horas. 12 52°. rain drop 0 % · 13 54°. rain ... Internacional América del Sur Argentina Ciudad Autónoma de Buenos Aires Buenos Aires', 'score': 0.80070794}, {'title': 'Tiempo en Buenos Aires. Clima a 14 días', 'url': 'https://www.meteored.com.ar/tiempo-en_Buenos+Aires-America+Sur-Argentina-Ciudad+Autonoma+de+Buenos+Aires-SABE-1-13584.html', 'content': '|  | | * Lluvia  Lluvia  **0% 0 mm** * Humedad  Humedad **56%** * Punto de rocío  Punto de rocío  **4 °C** * Nubosidad  Nubosidad **28%**  * Sensación Térmica  Sensación Térmica  **13 °C** * Visibilidad  Visibilidad **50 km** * Viento  Viento - Medio **12 km/h** * Presión  Presión  **1020 hPa**  * Niebla  Ni

{'input': 'Puedes usar la herramienta tavily para buscar cual es la temperatura en buenos aires argentina?',
 'output': 'La temperatura en Buenos Aires, Argentina es de 12°C y la sensación térmica es de 13°C. El cielo está parcialmente nuboso.\n'}

## RAG

**RAG**: Retrieval Augmented Generation
Componentes:
- *chunks*: fragmentos de texto en los que se divide el archivo a procesar
- *embeddings*: representacion numerica de lo que contiene cada chunk
- *Chroma*: base de datos vectorial diseñada específicamente para almacenar, gestionar y buscar embeddings
- *consulta*: se vuelve una representacion numerica
- *retrieval*: la consulta se compara con los embeddings y se obtienen los chunks relevantes

Doc gemini embeddings model: https://ai.google.dev/gemini-api/docs/embeddings

**Primer paso - Preparar los datos**
- generar chunks
- generar embebes y almacenarlos en la base de datos Chroma


Se realiza en archivo RAG_lab1 ya que el guardar los embebebs en la bd tiene cirta incompatibilidad con jupyter

**Segundo paso - Consultar**
- carga la base de datos
- compara a query contra los chunks
- obtiene los top x mas parecidos


Se realiza en archivo RAG_lab1b 

# Ecosistema

## LangGraph

**LangGraph**
Es una biblioteca para construir app multifactor y con estado utilizando llm
- Puede cordinar multiples chain o agents a travea sw multiples etpadas
- Puede implementar comportamientos, tomar decisiones 

## LangSmith

**LangSmith**: Plataforma unificada para depurar, probar y monitorear aplicaciones de modelos de lenguejas y agentes inteligentes
- gratis para un unico usuario
- https://www.langchain.com/pricing-langsmith

## Hub

**Hub** Prompts de la comunidad 

## LangServe

**LangServe**: Facilita el despliegue y mantenimiento de app, convierte apps de ml en servidores API
- construido con FastAPI
- Se registran todas las entradas y salidas del servidor 

# Modelos

## Antrophic Claude

Anthropic models: 
- https://docs.anthropic.com/en/docs/models-overview 
- Pago

In [18]:
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-opus-latest")
promt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("user", "{input}"),
    ]
)
chain = promt | model
chain.invoke({"input": "Tell me a funny joke about Python."}).content


BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.'}}

## OpenAI

- Pago
- Chat Model Documents: https://python.langchain.com/v0.2/docs/integrations/chat/
- OpenAI Chat Model Documents: https://python.langchain.com/v0.2/docs/integrations/chat/openai/

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

promt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("user", "{input}"),
    ]
)
chain = promt | model
chain.invoke({"input": "Tell me a funny joke about Python."}).content


## Google - Gemini

- Gratuito con limitaciones 
- https://console.cloud.google.com/gen-app-builder/engines
- https://ai.google.dev/gemini-api/docs/models/gemini

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
promt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("user", "{input}"),
    ]
)
chain = promt | llm
chain.invoke({"input": "Tell me a funny joke about Python."}).content

"Why did the Python say goodbye to the C++ programmer?\n\nBecause he didn't want to get bitten by a class!"

## IBM Granite, Flan-T5, Mixtral

- paquete Python langchain-ibm. 
- Clases como WatsonxLLM y ChatWatsonx te permiten incorporar los modelos fundacionales de IBM (como los modelos Granite, Flan-T5, Mixtral, etc.) 
- Modelos de Embeddings: WatsonxEmbeddings te permite generar embeddings de texto usando los modelos de IBM, lo cual es crucial para aplicaciones de RAG (Generación Aumentada por Recuperación) y búsqueda semántica.
Rerankers: WatsonxRerank ofrece capacidades para reordenar los resultados de búsqueda, mejorando la relevancia en los flujos de trabajo de RAG.
- Toolkits: WatsonxToolkit proporciona un conjunto de herramientas diseñadas específicamente para interactuar con los servicios de watsonx dentro de los agentes de LangChain.

In [105]:
from langchain.prompts import PromptTemplate

# Puedes incluir variables como {input}, {history}, {tools}
custom_prompt2 = PromptTemplate.from_template("""
Eres un asistente muy útil. Usa tus herramientas, en cuanto tengas una respuesta lo comunicas y finalizas.

Historial de conversación:
{chat_history}

Herramientas disponibles: {tools}
                                             {agent_scratchpad}
                                             {tool_names}

Usuario: {input}
Asistente:
""")
custom_prompt = ChatPromptTemplate.from_messages([
    ("system", """         
     Eres un asistente de IA muy útil y amigable. Responde siempre de forma clara y concisa.        
      Utiliza las herramientas disponibles para obtener la información necesaria.Herramientas disponibles:  {tool_names} {tools}
     Historial de conversación: {chat_history}   
     {agent_scratchpad}    
     Cuando tengas la respuesta final a la pregunta del usuario o hayas completado la tarea,         
     responde usando el siguiente formato especial:         
     Final Answer: [Tu respuesta final aquí]         """),         
    MessagesPlaceholder(variable_name="chat_history"),         
    ("human", "{input}"),         
    ])
 


In [99]:
from langchain.agents import Tool

def sumar(a, b):
    return str(int(a) + int(b))

tools = [
    Tool(
        name="Sumador",
        func=lambda x: sumar(*x.split()),
        description="Suma dos números dados separados por un espacio. Ejemplo: '3 4'"
    )
]


In [100]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [108]:
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

agent = create_react_agent(
    tools=tools,
    llm=llm,
    prompt= custom_prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory = memory,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True
)


In [110]:
respuesta = agent_executor.invoke(
                    {"input": "¿Puedes sumar 3 5?"}
                )

print(respuesta)




> Entering new AgentExecutor chain...
Final Answer: 8


> Finished chain.
{'input': '¿Puedes sumar 3 5?', 'chat_history': [HumanMessage(content='¿Puedes sumar 3 5?', additional_kwargs={}, response_metadata={}), AIMessage(content='8', additional_kwargs={}, response_metadata={}), HumanMessage(content='¿Puedes sumar 3 5?', additional_kwargs={}, response_metadata={}), AIMessage(content='8', additional_kwargs={}, response_metadata={})], 'output': '8', 'intermediate_steps': []}


c:\Users\Socius\Desktop\Langchain\langchain practica\.venv\Lib\site-packages\langchain\memory\chat_memory.py:55: UserWarning: 'ConversationBufferMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(
